In [6]:
import sys
import pandas as pd
import heapq
from math import radians, cos, sin, asin, sqrt, tanh, inf

In [7]:
def find_eline(start_point, end_point):
    #print(start_point, end_point)
    a = (end_point[0] - start_point[0])/(end_point[1] - start_point[1])
    b = end_point[0] - a * end_point[1]
    return (a,b)

In [8]:
def eline_distance(start_point, end_point, city):
    #y=ax+b
    #distance from (x1,y1) to line = |y1 - mx1 - b|/(1 + m^2)
    (a,b) = find_eline(start_point,end_point)
    distance = abs(city[0] - a*city[1] - b)/math.sqrt(1 + math.pow(a,2))
    return distance

    


In [9]:
data = pd.read_csv("road-segments.txt", header=None, delimiter=r"\s+")
data = data.rename({0: 'city_1', 1: 'city_2', 2: 'length', 3: 'speed_limit', 4: 'highway'}, axis=1)
data = pd.DataFrame(data)

data_gps = pd.read_csv("city-gps.txt", header=None, delimiter=r"\s+")
data_gps = data_gps.rename({0: 'city', 1: 'lat', 2: 'long'}, axis=1)
data_gps = pd.DataFrame(data_gps)

In [10]:
start_city = "Martinsville,_Indiana"
end_city = "Indianapolis,_Indiana"

In [11]:
city_1s = data.city_1.unique()
city_2s = data.city_2.unique()
city_1s = list(city_1s)
city_2s = list(city_2s)
cities = city_1s + city_2s
states = [i.split(',')[1] for i in cities]
states = list(set(states))
states = [i.replace('_',' ')[1:] for i in states if '&'not in i]
states_real = ['Alabama','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']
states_filt = [i for i in states if i in states_real]

#pseudo code
create a state column
remove rows with unwanted states



In [12]:
data.sample()

,city_1,city_2,length,speed_limit,highway
7367,"Jackson,_Ohio","McArthur,_Ohio",18,45,OH_93


In [13]:
def state(row):
    state = row.split(',')[1]
    state = state.replace('_',' ')[1:]
    return state
data['state_c1'] = data['city_1'].apply(state)
data['state_c2'] = data['city_2'].apply(state)


In [14]:
data = data[data.state_c1.isin(states_filt)]
data = data[data.state_c2.isin(states_filt)]
data_gps['state'] = data_gps['city'].apply(state)
data_gps = data_gps[data_gps.state.isin(states_filt)]
gps_states = data_gps.state.unique()
state_gps_dict = {}
for state in gps_states:
    all_coords = list(data_gps.loc[data_gps.state == state,['long','lat']].values)
    all_coords = [list(i) for i in all_coords]
    state_gps_dict[state] = all_coords
visited_states = []
start_state = start_city.split(',')[1].replace('_',' ')[1:]
visited_states.append(start_state)

In [15]:
def haversine(lon1, lat1, lon2, lat2): ## Wikipedia https://en.wikipedia.org/wiki/Haversine_formula
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat2 = radians(lat2)
    # haversine formula 
    lon_diff = lon2 - lon1 
    lat_diff = lat2 - lat1 
    hav_theta = sin(lat_diff/2)**2 + cos(lat1) * cos(lat2) * sin(lon_diff/2)**2
    archav = 2 * asin(sqrt(hav_theta)) 
    r = 3958.5
    return archav * r
def get_accurate_gps(city,gps_df,dist_df):
    gps_cities = set(gps_df.city.unique())
    if city in gps_df.city:
        return gps_df.loc[gps_df.city == city,['long','lat']].values[0]
    else:
        temp = dist_df[(dist_df.city_1 == city) | (dist_df.city_2 == city)].sort_values(['length'])
        for index, row in temp.iterrows():
            near_city_col = 'city_2'
            if row['city_2'] == city:
                near_city_col = 'city_1'
            near_city = row[near_city_col]
            near_dist = row['length']
            if near_city in gps_cities:
                break
    if gps_df.loc[gps_df.city == near_city,['long','lat']].shape[0] < 1:
        return [0,0]
    return gps_df.loc[gps_df.city == near_city,['long','lat']].values[0]

In [16]:
data_gps.loc[data_gps.city == start_city,['long','lat']].values[0]

array([-86.4283279,  39.4278253])

In [17]:
import math
def best_cities(states,state_gps, data_gps, data):
    cities = []
    for state in states:
        min_dist = math.inf
        for city in state_gps[state]:
            #city_coods = get_accurate_gps(city, data_gps, data)
            start_coods = get_accurate_gps(start_city, data_gps, data)
            end_coods = get_accurate_gps(end_city, data_gps, data)
            dist = eline_distance(start_coods,end_coods, city)
            if dist < min_dist:
                min_dist = dist
                min_result = city
        cities.append((state, min_result))
    return cities

In [18]:
data = pd.read_csv("road-segments.txt", header=None, delimiter=r"\s+")
data = data.rename({0: 'city_1', 1: 'city_2', 2: 'length', 3: 'speed_limit', 4: 'highway'}, axis=1)
data = pd.DataFrame(data)

data_gps = pd.read_csv("city-gps.txt", header=None, delimiter=r"\s+")
data_gps = data_gps.rename({0: 'city', 1: 'lat', 2: 'long'}, axis=1)
data_gps = pd.DataFrame(data_gps)
start_city = "Martinsville,_Indiana"
end_city = "Indianapolis,_Indiana"
city_1s = data.city_1.unique()
city_2s = data.city_2.unique()
city_1s = list(city_1s)
city_2s = list(city_2s)
cities = city_1s + city_2s
states = [i.split(',')[1] for i in cities]
states = list(set(states))
states = [i.replace('_',' ')[1:] for i in states if '&'not in i]
states_real = ['Alabama','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']
states_filt = [i for i in states if i in states_real]
def state(row):
    state = row.split(',')[1]
    state = state.replace('_',' ')[1:]
    return state
data['state_c1'] = data['city_1'].apply(state)
data['state_c2'] = data['city_2'].apply(state)
data = data[data.state_c1.isin(states_filt)]
data = data[data.state_c2.isin(states_filt)]
data_gps['state'] = data_gps['city'].apply(state)
data_gps = data_gps[data_gps.state.isin(states_filt)]
gps_states = data_gps.state.unique()
state_gps_dict = {}
for state in gps_states:
    all_coords = list(data_gps.loc[data_gps.state == state,['long','lat']].values)
    all_coords = [list(i) for i in all_coords]
    state_gps_dict[state] = all_coords
visited_states = []
start_state = start_city.split(',')[1].replace('_',' ')[1:]
visited_states.append(start_state)

In [19]:

roam_list = best_cities(states_filt, state_gps_dict, data_gps, data)

In [20]:
! pip install tsp_solver2

In [21]:
#len(roam_list)
from tsp_solver.greedy import solve_tsp
D = [[],
     [1.0],
     [2.0, 3.0]]


path = solve_tsp( D )

#will print [1,0,2], path with total length of 3.0 units
print(path)

[1, 0, 2]


In [38]:
roam_list
D_matrix = []
for i in range(len(roam_list)):
    D_matrix.append([])
    a_index = roam_list[i]
    state_a,coords_a = a_index[0],a_index[1]
    for j in range(len(roam_list)):
        if j<i:
            b_index = roam_list[j]
            state_b,coords_b = b_index[0],b_index[1]
            dist = haversine(*coords_a,*coords_b)
            # print(dist)
            D_matrix[i].append(dist)

In [39]:
D_matrix

[[],
 [223.04810721075086],
 [743.8765884875779, 629.4539236351457],
 [873.1034284930234, 672.1149377677383, 423.20139782071755],
 [956.8575601600223,
  1145.659569777873,
  1684.1605530232005,
  1817.7657404170843],
 [236.11705843734777,
  456.4577876079613,
  938.937209355573,
  1106.4310517577162,
  745.2233713262253],
 [1547.6575466126503,
  1373.893436774476,
  851.4316874298095,
  729.5417117171888,
  2504.456591901569,
  1766.0789169900872],
 [449.79347280387447,
  244.45153294323768,
  454.02185301800444,
  427.93225116566015,
  1389.916128738197,
  685.5035643312945,
  1133.8091934196482],
 [1644.419806419554,
  1426.4302361774678,
  1171.6182110507932,
  803.4679585875829,
  2546.3276765855794,
  1880.5032213771883,
  676.2819309821285,
  1197.1124200518686],
 [526.6096357015476,
  505.54620129601466,
  313.13641728033843,
  667.6804856326585,
  1413.249589385875,
  676.4092984234853,
  1161.9110945771308,
  460.67132579771544,
  1458.6777899947506],
 [616.4652908701011,
  80

In [56]:
start_state = start_city.split(',')[1].replace('_',' ')[1:]
end_state = end_city.split(',')[1].replace('_',' ')[1:]
start_state_index = [i for i in range(len(roam_list)) if roam_list[i][0] == start_state]
end_state_index = [i for i in range(len(roam_list)) if roam_list[i][0] == end_state]

In [58]:
path = solve_tsp( D_matrix,endpoints=(start_state_index[0],end_state_index[0]) )

# Given path:
start from 1st element \
find the distance to there \
when reached go to the next one \
till you reach end_state 

In [ ]:
def travel_to_state(curr_city,dest_state,dest_gps,data,data_gps):
    # make sure you take the whole data df and not the partial
    curr_city_gps = get_accurate_gps(curr_city,data_gps,data)
    cond = (data[data.city_1 == curr_city]) | (data[data.city_2==curr_city])
    for index,row in data[cond].iterrows():
        near_city_col = 'city_2'
        if row['city_2'] == curr_city:
            near_city_col = 'city_1'
        near_city = row[near_city_col]
        near_city_gps = get_accurate_gps(near_city,data_gps,data)
        cost = haversine(*dest_gps,*near_city_gps) + haversine(*near_city_gps,*curr_city_gps)

    return True

In [75]:
start_city

'Martinsville,_Indiana'

In [66]:
curr_city = start_city
dest_gps = roam_list[path[1]][1]
data_og = pd.read_csv("road-segments.txt", header=None, delimiter=r"\s+")
data_og = data_og.rename({0: 'city_1', 1: 'city_2', 2: 'length', 3: 'speed_limit', 4: 'highway'}, axis=1)

data_gps_og = pd.read_csv("city-gps.txt", header=None, delimiter=r"\s+")
data_gps_og = data_gps_og.rename({0: 'city', 1: 'lat', 2: 'long'}, axis=1)

In [84]:
state_end_name = data_gps_og.loc[(data_gps_og.long == dest_gps[0])&(data_gps_og.lat == dest_gps[1]),'city'].values[0]

In [99]:
curr_city = start_city
total_costs = [[],0,0,0]
for i in range(len(path[:2])):
    dest_gps = roam_list[path[i+1]][1]
    state_name = roam_list[path[i]][0]
    state_coords = roam_list[path[i]][1]
    state_end_name = data_gps_og.loc[(data_gps_og.long == dest_gps[0])&(data_gps_og.lat == dest_gps[1]),'city'].values[0]
    costs_state = distance(curr_city,state_end_name,data_og,data_gps_og)
    curr_city = costs_state[0][-1][0]
    total_costs[0].extend(costs_state[0])
    for j in range(1,len(costs_state)):
        total_costs[j]+=costs_state[j]

In [100]:
total_costs

[[('Jct_I-465_&_IN_37_S,_Indiana', 'IN_37'),
  ('Jct_I-65_&_I-465_S,_Indiana', 'I-74/465'),
  ('Jct_I-74_&_I-465_E,_Indiana', 'I-74/465'),
  ('Jct_I-465_&_US_52,_Indiana', 'I-465'),
  ('Jct_I-70_&_I-465_E,_Indiana', 'I-465'),
  ('Spiceland,_Indiana', 'I-70'),
  ('Richmond,_Indiana', 'I-70'),
  ('Vandalia,_Ohio', 'I-70'),
  ('Fairborn,_Ohio', 'I-70'),
  ('Springfield,_Ohio', 'I-70'),
  ('Lafayette,_Ohio', 'I-70'),
  ('New_Rome,_Ohio', 'I-70'),
  ('Columbus,_Ohio', 'I-70'),
  ('Reynoldsburg,_Ohio', 'I-70'),
  ('Kirkersville,_Ohio', 'I-70'),
  ('Zanesville,_Ohio', 'I-70'),
  ('Cambridge,_Ohio', 'I-70'),
  ('Hendrysburg,_Ohio', 'I-70'),
  ('St._Clairsville,_Ohio', 'I-70'),
  ('Bellaire,_Ohio', 'I-470'),
  ('Bethlehem,_West_Virginia', 'I-470'),
  ('Triadelphia,_West_Virginia', 'I-470'),
  ('Washington,_Pennsylvania', 'I-70'),
  ('Wickhaven,_Pennsylvania', 'I-70'),
  ('New_Stanton,_Pennsylvania', 'I-70'),
  ('Greensburg,_Pennsylvania', 'US_119'),
  ('New_Alexandria,_Pennsylvania', 'US_119'),

In [87]:
def cost(dist,time,delivery,row,col_to_cons,curr_city):
    dist[row[col_to_cons]] = dist[curr_city] + row['length']
    time[row[col_to_cons]] = time[curr_city] + row['length']/row['speed_limit'] 
    delivery[row[col_to_cons]] = delivery[curr_city] + row['length']/row['speed_limit'] 
    if row['speed_limit']>= 50:
        delivery[row[col_to_cons]] += (tanh(row['length']/1000)*2*(time[curr_city] + row['length']/row['speed_limit']))
    return (dist,time,delivery)
def distance(start, end,data,data_gps): # if end lat long not available, if mid lat long not available
    visited = dict() # making a dictionary for marking visited nodes 
    dist = dict({start:0})
    time = dict({start:0})
    delivery = dict({start:0})
    queue = []
    path = []
    parent = dict()
    
    #path.append(start)

    # cond_gps = (data_gps['city'] == end)
    #end_gps = data_gps[cond_gps]
    # end_index = data_gps.index[cond_gps]
    
    end_coords = get_accurate_gps(end,data_gps,data)
    parent[start] = ("-1", "-1")
    heapq.heappush(queue,(0,0,0, start))
    while(queue):
        curr_dist,curr_time,curr_delivery, curr_city = heapq.heappop(queue)
        if(curr_city == end):
            break;
        if(curr_city in visited):
            continue
        # print(curr_city)  
        visited[curr_city] = True
            
        cond = (data['city_1'] == curr_city) | (data['city_2'] == curr_city)
        # cond2 = (data['city_2'] == curr_city)
        for index, row in data[cond].iterrows():
            col_to_cons = 'city_1'
            if row['city_1'] == curr_city:
                col_to_cons = 'city_2'
            #print(row['city_1'], row['city_2'])
            if(row[col_to_cons] not in dist):
                dist,time,delivery = cost(dist,time,delivery,row,col_to_cons,curr_city)
                parent[row[col_to_cons]] = (curr_city, row['highway'])
            else:
                if(dist[row[col_to_cons]] > dist[curr_city] + row['length']):
                    dist,time,delivery = cost(dist,time,delivery,row,col_to_cons,curr_city)
                    parent[row[col_to_cons]] = (curr_city, row['highway'])

            # cond1_gps = (data_gps['city'] == row[col_to_cons])
            mid_city = row[col_to_cons]
            mid_coords = get_accurate_gps(mid_city,data_gps,data)
            haversine_dist = haversine(end_coords[0],end_coords[1], mid_coords[0],mid_coords[1])
            
            heapq.heappush(queue, (dist[row[col_to_cons]] + haversine_dist,
                        time[row[col_to_cons]] + haversine_dist/row['speed_limit'],
                        delivery[row[col_to_cons]]+ haversine_dist/row['speed_limit'] ,
                        row[col_to_cons] ) )
    
    tmp = end
    curr_path = []
    while(tmp != start):
        tmp1 = tmp;
        tmp, hw = parent[str(tmp)]
        curr_path.append((str(tmp1), hw))
        
    return curr_path[::-1],dist[end], time[end], delivery[end]

(453, 7.772027972027973, 10.693722998046285)